# Text Translation and Sentiment Analysis using Transformers

## Project Overview:

The objective of this project is to analyze the sentiment of movie reviews in three different languages - English, French, and Spanish. We have been given 30 movies, 10 in each language, along with their reviews and synopses in separate CSV files named `movie_reviews_eng.csv`, `movie_reviews_fr.csv`, and `movie_reviews_sp.csv`.

- The first step of this project is to convert the French and Spanish reviews and synopses into English. This will allow us to analyze the sentiment of all reviews in the same language. We will be using pre-trained transformers from HuggingFace to achieve this task.

- Once the translations are complete, we will create a single dataframe that contains all the movies along with their reviews, synopses, and year of release in all three languages. This dataframe will be used to perform sentiment analysis on the reviews of each movie.

- Finally, we will use pretrained transformers from HuggingFace to analyze the sentiment of each review. The sentiment analysis results will be added to the dataframe. The final dataframe will have 30 rows


The output of the project will be a CSV file with a header row that includes column names such as **Title**, **Year**, **Synopsis**, **Review**, **Review Sentiment**, and **Original Language**. The **Original Language** column will indicate the language of the review and synopsis (*en/fr/sp*) before translation. The dataframe will consist of 30 rows, with each row corresponding to a movie.

In [1]:
# imports
import pandas as pd
from transformers import MarianMTModel, MarianTokenizer
from transformers import pipeline

### Get data from `.csv` files and then preprocess data

In [2]:
# use the `pd.read_csv()` function to read the movie_review_*.csv files into 3 separate pandas dataframes

# Note: All the dataframes would have different column names. For testing purposes
# you should have the following column names/headers -> [Title, Year, Synopsis, Review]

def preprocess_data() -> pd.DataFrame:
    """
    Reads movie data from .csv files, map column names, add the "Original Language" column,
    and finally concatenate in one resultant dataframe called "df".
    """    
    columns = ['Title', 'Year', 'Synopsis', 'Review']

    # Load csv files and rename columns to english
    df_eng = pd.read_csv("data/movie_reviews_eng.csv", names=columns, header=0)
    df_fr = pd.read_csv("data/movie_reviews_fr.csv", names=columns, header=0)
    df_sp = pd.read_csv("data/movie_reviews_sp.csv", names=columns, header=0)

    # Add a new column Language
    df_eng['Original Language'] = 'en'
    df_fr['Original Language'] = 'fr'
    df_sp['Original Language'] = 'sp'

    # Concatenate all
    df_combined = pd.concat([df_eng, df_fr, df_sp], ignore_index=True)
    return df_combined

df = preprocess_data()

In [3]:
df.sample(10)

,Title,Year,Synopsis,Review,Original Language
11,Intouchables,2011,Ce film raconte l'histoire de l'amitié improba...,"""Intouchables est un film incroyablement touch...",fr
28,Torrente: El brazo tonto de la ley,1998,"En esta comedia española, un policía corrupto ...","""Torrente es una película vulgar y ofensiva qu...",sp
9,The Island,2005,In a future where people are cloned for organ ...,"""The Island is a bland and forgettable sci-fi ...",en
1,The Dark Knight,2008,Batman (Christian Bale) teams up with District...,"""The Dark Knight is a thrilling and intense su...",en
6,Scott Pilgrim vs. the World,2010,Scott Pilgrim (Michael Cera) must defeat his n...,"""It was difficult to sit through the whole thi...",en
10,La La Land,2016,Cette comédie musicale raconte l'histoire d'un...,"""La La Land est un film absolument magnifique ...",fr
19,Babylon A.D.,2008,"Dans un futur lointain, un mercenaire doit esc...","""Ce film est un gâchis complet. Les personnage...",fr
16,La Tour Montparnasse Infernale,2001,Deux employés de bureau incompétents se retrou...,"""Je ne peux pas croire que j'ai perdu du temps...",fr
3,The Godfather,1972,Don Vito Corleone (Marlon Brando) is the head ...,"""The Godfather is a classic movie that stands ...",en
18,Les Visiteurs en Amérique,2000,Dans cette suite de la comédie française Les V...,"""Le film est une perte de temps totale. Les bl...",fr


### Text translation

Translate the **Review** and **Synopsis** column values to English.

In [5]:
# load translation models and tokenizers
fr_en_model_name = "Helsinki-NLP/opus-mt-fr-en"
es_en_model_name = "Helsinki-NLP/opus-mt-es-en"
fr_en_model = MarianMTModel.from_pretrained(fr_en_model_name)
es_en_model = MarianMTModel.from_pretrained(es_en_model_name)
fr_en_tokenizer = MarianTokenizer.from_pretrained(fr_en_model_name)
es_en_tokenizer = MarianTokenizer.from_pretrained(es_en_model_name)

def translate(text: str, model, tokenizer) -> str:  
    """
    Function to translate a text using a pre-trained model and tokenizer.
    
    Parameters:
    text (str): The input text to be translated.
    model: The pre-trained translation model from Hugging Face.
    tokenizer: The corresponding tokenizer for the translation model.
    
    Returns:
    str: The translated text.
    """
    # Encode the text using the tokenizer. 
    # This converts the text into token IDs that the model can process.
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)

    # Generate the translation using the model.
    # The model generates output token IDs which correspond to the translated text.
    outputs = model.generate(**inputs)

    # Decode the generated output to convert the token IDs back into a human-readable string.
    decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    return decoded

In [6]:
# Filter reviews in French and translate to English
fr_reviews = df[df['Original Language'] == 'fr']['Review']
fr_reviews_en = fr_reviews.apply(lambda x: translate(x, fr_en_model, fr_en_tokenizer))

# Filter synopsis in French and translate to English
fr_synopsis = df[df['Original Language'] == 'fr']['Synopsis']
fr_synopsis_en = fr_synopsis.apply(lambda x: translate(x, fr_en_model, fr_en_tokenizer))

# Filter reviews in Spanish and translate to English
es_reviews = df[df['Original Language'] == 'sp']['Review']
es_reviews_en = es_reviews.apply(lambda x: translate(x, es_en_model, es_en_tokenizer))

# Filter synopsis in Spanish and translate to English
es_synopsis = df[df['Original Language'] == 'sp']['Synopsis']
es_synopsis_en = es_synopsis.apply(lambda x: translate(x, es_en_model, es_en_tokenizer))

# Update dataframe with translated text
# Overwrite the existing data with translated reviews and synopsis
df.loc[df['Original Language'] == 'fr', 'Review'] = fr_reviews_en
df.loc[df['Original Language'] == 'fr', 'Synopsis'] = fr_synopsis_en
df.loc[df['Original Language'] == 'sp', 'Review'] = es_reviews_en
df.loc[df['Original Language'] == 'sp', 'Synopsis'] = es_synopsis_en

In [7]:
df.sample(10)

,Title,Year,Synopsis,Review,Original Language
29,El Incidente,2014,"In this Mexican horror film, a group of people...","""The Incident is a boring and frightless film ...",sp
27,El Bar,2017,A group of people are trapped in a bar after M...,"""The Bar is a ridiculous and meaningless film ...",sp
11,Intouchables,2011,This film tells the story of the unlikely frie...,"""Untouchables is an incredibly touching film w...",fr
24,Amores perros,2000,Three stories intertwine in this Mexican film:...,"""Amores dogs is an intense and moving film tha...",sp
25,Águila Roja,(2009-2016),This Spanish television series follows the adv...,"""Red Eagle is a boring and uninteresting serie...",sp
16,La Tour Montparnasse Infernale,2001,Two incompetent office workers find themselves...,"""I can't believe I've wasted time watching thi...",fr
17,Astérix aux Jeux Olympiques,2008,In this film adaptation of the popular comic s...,"""This film is a complete disappointment. The j...",fr
14,Le Fabuleux Destin d'Amélie Poulain,2001,This romantic comedy tells the story of Amélie...,"""The Fabulous Destiny of Amélie Poulain is an ...",fr
21,La Casa de Papel,(2017-2021),This Spanish television series follows a group...,"""The Paper House is an exciting and addictive ...",sp
2,Forrest Gump,1994,Forrest Gump (Tom Hanks) is a simple man with ...,"""Forrest Gump is a heartwarming and inspiratio...",en


### Sentiment Analysis

Use HuggingFace pretrained model for sentiment analysis of the reviews. Store the sentiment result **Positive** or **Negative** in a new column titled **Sentiment** in the dataframe.

I know there are other sentiment analyzers which return directly positive, neutral and negative, but I wanted to try it with that model. Thus I convert the labels (1 to 5 Stars to Negative, Neutral and Positive). 

Other possible models would be (Examples):

- distilbert-base-uncased-finetuned-sst-2-english
- cardiffnlp/twitter-roberta-base-sentiment
- siebert/sentiment-roberta-large-english

In [24]:
# Load sentiment analysis model
model_name = "nlptown/bert-base-multilingual-uncased-sentiment"
sentiment_classifier = pipeline("sentiment-analysis", model=model_name)

def analyze_sentiment(text, classifier):
    """
    Function to perform sentiment analysis on a text using a model.
    
    Parameters:
    text (str): The input text to analyze.
    classifier: The sentiment analysis model pipeline.
    
    Returns:
    str: The sentiment label (e.g., 'Positive', 'Negative', 'Neutral').
    """
    # Perform sentiment analysis
    result = classifier(text)[0]  # The result is a list, we take the first element

    # Map star ratings to sentiment categories
    star_to_sentiment = {
        "1 star": "Negative",
        "2 stars": "Negative",
        "3 stars": "Neutral",
        "4 stars": "Positive",
        "5 stars": "Positive"
    }
    
    # Extract the label (e.g., '1 star', '2 stars') and map it to 'Negative', 'Neutral', or 'Positive'
    sentiment = star_to_sentiment.get(result['label'], "Neutral")

    return sentiment

/home/oliver/anaconda3/envs/pytorch_env/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [21]:
# perform sentiment analysis on reviews and store results in new column
df['Sentiment'] = df['Review'].apply(lambda x: analyze_sentiment(x, sentiment_classifier))

In [23]:
df.sample(30)

,Title,Year,Synopsis,Review,Original Language,Sentiment
25,Águila Roja,(2009-2016),This Spanish television series follows the adv...,"""Red Eagle is a boring and uninteresting serie...",sp,Negative
14,Le Fabuleux Destin d'Amélie Poulain,2001,This romantic comedy tells the story of Amélie...,"""The Fabulous Destiny of Amélie Poulain is an ...",fr,Positive
0,The Shawshank Redemption,1994,"Andy Dufresne (Tim Robbins), a successful bank...","""The Shawshank Redemption is an inspiring tale...",en,Positive
29,El Incidente,2014,"In this Mexican horror film, a group of people...","""The Incident is a boring and frightless film ...",sp,Negative
22,Y tu mamá también,2001,Two teenage friends (Gael García Bernal and Di...,"""And your mom is also a movie that stays with ...",sp,Positive
5,Blade Runner 2049,2017,"Officer K (Ryan Gosling), a new blade runner f...","""Boring and too long. Nothing like the origina...",en,Negative
11,Intouchables,2011,This film tells the story of the unlikely frie...,"""Untouchables is an incredibly touching film w...",fr,Positive
3,The Godfather,1972,Don Vito Corleone (Marlon Brando) is the head ...,"""The Godfather is a classic movie that stands ...",en,Positive
20,Roma,2018,Cleo (Yalitza Aparicio) is a young domestic wo...,"""Rome is a beautiful and moving film that pays...",sp,Positive
24,Amores perros,2000,Three stories intertwine in this Mexican film:...,"""Amores dogs is an intense and moving film tha...",sp,Positive


In [26]:
# export the results to a .csv file
df.to_csv("result/reviews_with_sentiment.csv", index=False)